In [1]:
from scipy import constants
from astropy import units as u
import numpy as np 

# SVS13

In [120]:
r=150*u.au
M_star = 0.41*u.solMass
M_disk = 0.22*u.solMass
T=100*u.K #disk temperature

In [122]:
r=300*u.au

# Tobin (HOPS 370)

In [18]:
r=50*u.au
M_star = 2.5*u.solMass
M_disk = 0.06*u.solMass
T=180*u.K #disk temperature

# Calculo M (dust)

In [26]:
S = 1102#en mJy
D = 300# en pc
k_nu = 1.77 #en cm^2/g (del polvo)
f= 100 # gas-to-dust ratio
T_d = np.array([126-23,126,126+23]) #en K (126+-23K)
M=5.3e-3*(S/10)*(D/140)**2*(k_nu/f/0.01)**(-1)*(T_d/15)**(-1) #en solar mass
print(M)

[0.22066054 0.18038123 0.15253715]


# Calculo N(H2)

In [64]:
d=0.6 #diameter in arcsec
D=300 #distance in pc
r=d/2*D*u.au #radio in AU 
A=np.pi*(r.to(u.cm))**2 #area of the disk
m_H2 = 2*1.00794*u.u #molecular mass (H_2)
N_H2=M*u.M_sun/(A*m_H2.to(u.M_sun)) 
print(N_H2)

[2.30160658e+25 1.88147205e+25 1.59104348e+25] 1 / cm2


# Calculo abundancia

In [65]:
N_Sp = 3.9e14*u.cm**(-2) #column density of the species
Ab_Sp = N_Sp/N_H2 #abundance of the species with respect to H2
print(Ab_Sp)

[1.69446856e-11 2.07284504e-11 2.45122151e-11]


# Calculo M

In [68]:
d=0.6 #diameter in arcsec
D=300 #distance in pc
N_Sp = 3.9e14*u.cm**(-2) #column density of the species
Frac_Ab_Sp = 1/45#fractional abundance of the species (13C/12C=1/45, 15N/14N=1/234 Crocket 2014 -> Tercero 2010)
Ab_Mol = 0.9e-9 #abundance of the molecule respect to H2 (HCN=3.8e-7 hot core, 4.1e-9 compact ridge, 2.8e-7 plateu, 3.3e-8 extendend ridge, Crocket 2014)
Ab_Sp = Frac_Ab_Sp * Ab_Mol #abundance of the species with respect to H2
r=d/2*D*u.au #radio in AU 
A=np.pi*(r.to(u.cm))**2 #area of the disk
m_H2 = 2*1.00794*u.u #molecular mass (H_2)
N_H2 = N_Sp/Ab_Sp
M_H2 = A*N_H2*m_H2.to(u.M_sun)
print(Ab_Sp)
print(N_H2)
print(M_H2)

2.0000000000000002e-11
1.9499999999999997e+25 1 / cm2
0.18695117121429097 solMass


# Calculo Q

In [19]:
G = constants.G * u.m**3 * u.kg**(-1) * u.s**(-2) #Gravitatcional constant
gamma=7/5 #adiabatic index, 7/5 for a diatomic gas and 5/3 for a monoatomic
k = constants.k * u.kg * u.m**2 * u.s**(-2) * u.K**(-1) #Boltzmann constant 
m_H2 = 2*1.00794*u.u #molecular mass (H_2)
#cs = (gamma*k*T/mH2.to(u.kg))**(1/2) #disk sound speed
cs=560*u.m*u.s**(-1)
v = np.sqrt(G*M_star.to(u.kg)/r.to(u.m)) #linear velocity at radius r
w = v/r.to(u.m) #Keplerian angular velocity
H = cs/w # disk scale height
Q = 2*M_star/M_disk * H/r.to(u.m) #(for a rotationally supported disk around a protostar, Kratter & Lodato 2016, Tobin 2016)
print(Q)

7.006944342195134
